In [ ]:
#%matplotlib notebook
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(9,4.5))
rc('font',size=12)
rc('text',usetex=False)

from qubicpack.qubicfp import qubicfp
import qubic.fibtools as ft
import qubic.plotters as p
import qubic.lin_lib as ll
import qubic.demodulation_lib as dl
import satorchipy as stpy
from pysimulators import FitsArray

import numpy as np
from matplotlib.pyplot import *
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import glob
import string
import scipy.signal as scsig
from scipy import interpolate
import datetime as dt
import pickle
from importlib import reload

# Setting the directory containing data

Here are the vailable data:
### 2020-01-14: 
- 2020-01-14_18.38.23__ScanFast_Speed_VE5_DeltaAz_40_DeltaEl_30_NScans_151_Cycle_0: 
    - Long acquisition during the night with SOurce Modulation 
    - Note that the computer experienced a reboot during the night (at ~5AM) due to some windows update probably. As a result the scan is not fully complete and STeve had to modify by hand some fits files in order to make them readable. After discussion with Wilfried, it seems that there is a solution to this kind of issues in the way QS files a re written and this will soon be implemented.
    - Edit (16/01/2020): Steve has corrected the dataset so in principle it can be read and analysed
    
### 2020-01-15: 
- 2020-01-15_21.34.32__ScanFast_Speed_VE10_DeltaAz_40_DeltaEl_30_NScans_151_Cycle_0:
    - Long acquisition overnight. It was intended to be without the modulation of the source but acttually the source was modulating in sine. So it should be analysed with the current cript instead of the one not using modulation.

### 2020-03-16: 
- 2020-03-16_10.55.31__ScanFast_Speed_VE10_DeltaAz_10_DeltaEl_10_NScans_1_Cycle_0:
    - Short test
- 2020-03-16_11.04.58__ScanFast_Speed_VE12_DeltaAz_50_DeltaEl_10_NScans_1_Cycle_0:
    - short test
- 2020-03-16_11.14.29__ScanFast_Speed_VE12_DeltaAz_50_DeltaEl_30_NScans_51_Cycle_0:
    - interrupted
- 2020-03-16_12.46.27__ScanFast_Speed_VE12_DeltaAz_50_DeltaEl_30_NScans_51_Cycle_0:
    - Full Cycle



In [ ]:
#day = '2020-03-16'
#keyword = '*ScanMap'
#day = '2022-04-14'
#keyword = '*ScanFast*'
day = '2022-04-23'
keyword = '*ScanMap*'
data_dir = '/qubic/Data/Calib-TD/'+day+'/'
#data_dir = '/Users/hamilton/Qubic/Calib-TD/'+day+'/'
dirs = np.sort(glob.glob(data_dir+keyword))

print (dirs)


In [ ]:
ifile = 3
thedir = dirs[ifile]
print(thedir)
a = qubicfp()
a.read_qubicstudio_dataset(thedir)

In [ ]:
print(a.hk.keys())

In [ ]:
print(a.hk['INTERN_HK'].keys())

In [ ]:
print(a.hk['EXTERN_HK'].keys())

In [ ]:
print(a.hk['CALSOURCE-CONF'].keys())

In [ ]:
rc('figure',figsize=(12,12))

time_azel = a.timeaxis(datatype='hk',axistype='pps')
az = a.azimuth()
el = a.elevation()

t_src = a.calsource()[0]
data_src = a.calsource()[1]

t0 = time_azel[0]

subplot(2,2,1)
plot((time_azel-t0)/3600, az,',')
xlabel('Time [h]')
ylabel('Az')
subplot(2,2,2)
plot((time_azel-t0)/3600, el,',')
xlabel('Time [h]')
ylabel('El')
ylim(30,70)

subplot(2,2,3)
plot(az*cos(np.radians(el)), el,',')
xlabel('Az')
ylabel('El')

subplot(2,2,4)
plot((t_src-t0)/3600, data_src,',')
xlabel('Time [h]')
ylabel('Src Data')


In [ ]:
TESNum = 95
asic = 1
tod = a.timeline(TES=TESNum,asic=asic)
tt = a.timeaxis(axistype='pps', asic=asic)

subplot(2,1,1)
plot((tt-t0)/3600, dl.renorm(tod), 
     label='Data TES {} ASIC {}'.format(TESNum,asic))
plot((t_src-t0)/3600, dl.renorm(data_src)-5, label='source')
xlabel('Hours')
legend(loc='upper left')
title(thedir)
ylim(-10,10)

subplot(2,1,2)
tzoom = 1.2940
dtzoom = 0.005
oktime_d = np.abs((tt-t0)/3600-tzoom) < dtzoom
oktime_s = np.abs((t_src-t0)/3600-tzoom) < dtzoom
plot((tt-t0)[oktime_d]/3600, dl.renorm(tod[oktime_d]), 
     label='Data TES {} ASIC {}'.format(TESNum,asic))
plot((t_src[oktime_s]-t0)/3600, dl.renorm(data_src[oktime_s]), label='source')
xlabel('Hours')
legend(loc='upper left')
title(thedir)


In [ ]:
rc('figure',figsize=(13,8))
#### Filtering
reload(ft)

thefreqmod = 1.

period = 1./ thefreqmod
lowcut = 0.5
highcut = 20

xmin = 0.01
xmax = 90.
ymin = 1e1
ymax = 1e17

############ Power spectrum
spectrum_f, freq_f = ft.power_spectrum(tt, tod, rebin=True)
plot(freq_f, f.gaussian_filter1d(spectrum_f,1),label='Raw Data')
yscale('log')
xscale('log')
xlabel('Frequency [Hz]')
ylabel('Power Spectrum')
xlim(xmin, xmax)
ylim(ymin, ymax)
title('TES {} ASIC {}'.format(TESNum,asic))

for i in range(10):
    plot([1./period*i,1./period*i],[ymin,ymax],'k--',alpha=0.3)

plot([lowcut, lowcut],[ymin,ymax],'k')
plot([highcut, highcut],[ymin,ymax],'k')
legend()

########## New Power spectrum
nharm = 10
notch = np.array([[1.724, 0.005, nharm]])
newdata = ft.filter_data(tt, tod, lowcut, highcut, notch=notch, rebin=True, verbose=True, order=5)
spectrum_f2, freq_f2 = ft.power_spectrum(tt, newdata, rebin=True)
plot(freq_f2, f.gaussian_filter1d(spectrum_f2,1),label='Filtered data')
for i in range(nharm):
    plot([notch[0,0]*(i+1),notch[0,0]*(i+1)], [ymin,ymax],'m:')
legend(loc='upper left')

tight_layout()

In [ ]:
plot((tt-t0)/3600, dl.renorm(newdata), 
     label='Data TES {} ASIC {}'.format(TESNum,asic))
plot((t_src-t0)/3600, dl.renorm(data_src)-5, label='source')
xlabel('Hours')
legend()
title(thedir)
ylim(-20,20)


In [ ]:
tzoom = 2.101
dtzoom = 0.003
oktime_d = np.abs((tt-t0)/3600-tzoom) < dtzoom
oktime_s = np.abs((t_src-t0)/3600-tzoom) < dtzoom
plot((tt-t0)[oktime_d]/3600, dl.renorm(newdata[oktime_d]), 
     label='Data TES {} ASIC {}'.format(TESNum,asic))
plot((t_src[oktime_s]-t0)/3600, dl.renorm(data_src[oktime_s]), label='source')
xlabel('Hours')
legend()
title(thedir)


Now we apply demodualtion with the source data

In [ ]:
reload(dl)
##### Filtering to be used
lowcut = 0.5
highcut = 70
nharm = 10
notch = np.array([[1.724, 0.005, nharm]])
fourier_cuts = [lowcut, highcut, notch]

TESNum = 96
asic = 1

tod = a.timeline(TES=TESNum,asic=asic)
tt = a.timeaxis(axistype='pps', asic=asic)

src = [tt, np.interp(tt, t_src, data_src)]
period = 1.
newt_demod, amp_demod, errors_demod = dl.demodulate_methods([tt, tod],
                                                            1./period, 
                                                            src_data_in=src,
                                                            method='demod_quad', remove_noise=True,
                                                           fourier_cuts=fourier_cuts)
tzoom = 2.101
dtzoom = 0.03
oktime_d = np.abs((tt-t0)/3600-tzoom) < dtzoom
plot((tt[oktime_d]-t0)/3600, (newdata[oktime_d]), 
     label='Data TES {} ASIC {}'.format(TESNum,asic))

okdemod = np.abs((newt_demod-t0)/3600-tzoom) < dtzoom
plot((newt_demod[okdemod]-t0)/3600, (amp_demod[okdemod]), label='demodulated')

xlabel('Hours')
legend()
title(thedir)


In [ ]:
oktime_azel = np.abs((time_azel-t0)/3600-tzoom) < dtzoom
print(el[oktime_azel])

In [ ]:
newaz = np.interp(newt_demod, time_azel, az)
newel = np.interp(newt_demod, time_azel, el)
azmin = -20
azmax = 20
elmin = 35
elmax = 65
naz = 100
nel = 51
mymap, azmap, elmap = dl.coadd_flatmap(amp_demod, newaz, newel, 
                                    filtering=None,
                                    azmin=azmin, azmax=azmax, 
                                    elmin=elmin, elmax=elmax, 
                                    naz=naz,nel=nel)


In [ ]:
imshow(mymap, aspect='equal',
       extent=[np.cos(np.radians(50))*azmin, np.cos(np.radians(50))*azmax, elmax, elmin])
title('TES {}'.format(TESNum))
xlabel('Cos(50) x Az')
ylabel('El')
colorbar()

In [ ]:
len(amp_demod)

Now loop on all TES and all ASICs

In [ ]:
reload(dl)
### Now do all maps with the same cuts
###
Period = 1.

##### Filtering to be used
lowcut = 0.5
highcut = 70
nharm = 10
notch = np.array([[1.724, 0.005, nharm]])
fourier_cuts = [lowcut, highcut, notch]

amps_demod = np.zeros((256, len(amp_demod)))

for asic in [1,2]:
    tt = a.timeaxis(axistype='pps', asic=asic)
    src = [tt, np.interp(tt, t_src, data_src)]
    for i in range(128):
        print('Mapmaking for Asic {} TES {}'.format(asic,i+1))    
        tod = a.timeline(TES=i+1,asic=asic)

        print('- Demodulation')
        newt_demod, amps_demod[i+128*(asic-1),:], errors_demod = dl.demodulate_methods([tt, tod],
                                                                    1./period, 
                                                                    src_data_in=src,
                                                                    method='demod_quad', remove_noise=True,
                                                                   fourier_cuts=fourier_cuts)


In [ ]:
azmin = -20
azmax = 20
elmin = 35
elmax = 65
naz = 51
nel = 51
nmaps = 256
mapdata = np.zeros((256, nel, naz))
newaz = np.interp(newt_demod, time_azel, az)
newel = np.interp(newt_demod, time_azel, el)

mapdata, azmap, elmap = dl.coadd_flatmap(-amps_demod, newaz, newel, 
                                                                filtering=None,
                                                                azmin=azmin, azmax=azmax, 
                                                                elmin=elmin, elmax=elmax, 
                                                                naz=naz,nel=nel)


In [ ]:
TESNum = 93
ites = TESNum-1
mm, ss = ft.meancut(mapdata[ites,:,:],3)
imshow(-mapdata[ites,:,:], aspect='equal',
       extent=[np.cos(np.radians(50))*azmin, np.cos(np.radians(50))*azmax, elmin, elmax],
       vmin=mm-ss, vmax=mm+30*ss)
title('TES {}'.format(ites+1))


In [ ]:
rc('figure',figsize=(16,8))
nnx = 5
nny=2
k=0
nplots = int(256/nnx/nny)+1


for k in range(nplots):
    figure()
    for i in range(nnx*nny):
        ites = i+k*nnx*nny
        if ites < 256:
            subplot(nny,nnx,i+1)
            mm, ss = ft.meancut(mapdata[ites,:,:],3)
            imshow(-mapdata[ites,:,:], aspect='equal',
                   extent=[np.cos(np.radians(50))*azmin, np.cos(np.radians(50))*azmax, elmin, elmax],
                   vmin=mm-ss, vmax=mm+30*ss)
            title('TES {}'.format(ites+1))
            savefig('toto{}.png'.format(k))
    tight_layout()
    show()

In [ ]:
TESNum = 213
mm, ss = ft.meancut(mapdata[TESNum-1,:,:],3)
imshow(-mapdata[TESNum-1,:,:], aspect='equal',
       extent=[np.cos(np.radians(50))*azmin, np.cos(np.radians(50))*azmax, elmin, elmax],
       vmin=mm-3*ss, vmax=mm+50*ss)
title('TES {}'.format(TESNum))


In [ ]:
az2d, el2d = np.meshgrid(azmap,elmap)

from mpl_toolkits.mplot3d import Axes3D

fig = figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(az2d, el2d, -mapdata[TESNum-1,:,:], cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)


In [ ]:
pwd